In [1]:
import yaml
from app.datasets.loader import load_test_cases
from app.datasets.validator import validate_dataset_schema
from app.datasets.preprocessing import preprocess_dataframe
from app.client.rag_client import RAGClient
from app.ragas.sample_builder import build_ragas_dataset
from app.ragas.evaluate import run_ragas_evaluation
from app.reporting.exporters import save_metrics

c:\Users\Usuario\Desktop\ragas_implementation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = load_test_cases('./app/data/raw/input.xlsx')
print(df)

   id                               user_input  \
0   1              Legalizar tirulo secundario   
1   2                           Turno para dni   
2   3       trámite renovación nicho chacarita   
3   4   copia acta de casamineto de mis padres   
4   5  Talleres y escuelas para discapacitados   

                                           reference  
0  Si querés **legalizar un título secundario** e...  
1  Acá te dejo la info para sacar el **DNI**. Pri...  
2  Acá te dejo la info para **renovar el arrendam...  
3  Si querés pedir una **copia del acta de matrim...  
4  En Buenos Aires hay actividades para personas ...  


In [5]:
df = validate_dataset_schema(df)
print(df)

   id                               user_input  \
0   1              Legalizar tirulo secundario   
1   2                           Turno para dni   
2   3       trámite renovación nicho chacarita   
3   4   copia acta de casamineto de mis padres   
4   5  Talleres y escuelas para discapacitados   

                                           reference  
0  Si querés **legalizar un título secundario** e...  
1  Acá te dejo la info para sacar el **DNI**. Pri...  
2  Acá te dejo la info para **renovar el arrendam...  
3  Si querés pedir una **copia del acta de matrim...  
4  En Buenos Aires hay actividades para personas ...  


In [6]:
df = preprocess_dataframe(df)
print(df)

   id                               user_input  \
0   1              Legalizar tirulo secundario   
1   2                           Turno para dni   
2   3       trámite renovación nicho chacarita   
3   4   copia acta de casamineto de mis padres   
4   5  Talleres y escuelas para discapacitados   

                                           reference  
0  Si querés **legalizar un título secundario** e...  
1  Acá te dejo la info para sacar el **DNI**. Pri...  
2  Acá te dejo la info para **renovar el arrendam...  
3  Si querés pedir una **copia del acta de matrim...  
4  En Buenos Aires hay actividades para personas ...  


In [7]:
with open('./app/config/config.yaml', 'r') as file:
  config_data = yaml.load(file, Loader= yaml.FullLoader)
  
print(config_data)
client = RAGClient(config_data)

{'agent': {'base_url': 'https://aca-orc-dev.happysmoke-87dca937.eastus2.azurecontainerapps.io/api/chat/test', 'timeout': 3, 'max_retries': 3}, 'headers': {'Content-Type': 'application/json'}, 'body': {'trace_id': ''}, 'llm': {'model': 'gpt-4o', 'embeddings': 'text-embedding-3-large', 'api_version': '2024-12-01-preview'}}


In [8]:
responses = client.query_batch(df['user_input'], df['reference'])

save_responses = client.save_api_responses(responses)

dataset = build_ragas_dataset(responses)

Processing queries: 100%|██████████| 5/5 [00:52<00:00, 10.41s/it]


In [26]:
results = run_ragas_evaluation(dataset, config_data)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  40%|████      | 10/25 [00:47<00:48,  3.26s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 25/25 [01:42<00:00,  4.09s/it]


{'semantic_similarity': 0.8105, 'answer_relevancy': 0.5007, 'context_precision': 0.9667, 'faithfulness': 0.9000, 'context_recall': 0.5971}


In [27]:
exported_results = save_metrics('./app/data/processed/results_outcome.json', results)
exported_results = save_metrics('./app/data/processed/results_outcome.xlsx', results)